## Spam Classification

In [1]:
%%loadFromPOM
<dependency>
    <groupId>org.apache.opennlp</groupId>
    <artifactId>opennlp-tools</artifactId>
    <version>1.9.0</version>
</dependency>

In [2]:
import java.io.BufferedReader;
import java.io.BufferedWriter;
import java.io.File;
import java.io.FileInputStream;
import java.io.FileNotFoundException;
import java.io.FileReader;
import java.io.FileWriter;
import java.io.IOException;
import java.io.InputStream;
import java.nio.charset.StandardCharsets;
import java.util.ArrayList;
import opennlp.tools.doccat.DoccatModel;
import opennlp.tools.doccat.DocumentCategorizerME;
import opennlp.tools.doccat.DocumentSample;
import opennlp.tools.doccat.DocumentSampleStream;
import opennlp.tools.util.ObjectStream;
import opennlp.tools.util.InputStreamFactory;
import opennlp.tools.util.PlainTextByLineStream;
import opennlp.tools.util.MarkableFileInputStreamFactory;

In [4]:
ArrayList<String> testList = new ArrayList();
try (BufferedWriter spamBufferedWriter = new BufferedWriter(
        new FileWriter(new File("../data/spamtraining.train")))) {
    
    String rootDirectoryName = "../data/lingspam_public/stop";
    File rootDirectory = new File(rootDirectoryName);
    for (String directoryName : rootDirectory.list()) {
        File file = new File(rootDirectoryName + "/" + directoryName);
        
        // Open one file at a time
        String fileNames[] = file.list();
        if (fileNames != null) {
            for (String fileName : fileNames) {
                String filePath = rootDirectoryName + "/" + directoryName + 
                    "/" + fileName;
                StringBuilder lineStringBuilder = new StringBuilder();
                
                // OpenNLP requires a SPAM or HAM header, followed by single line text 
                BufferedReader br = new BufferedReader(new FileReader(new File(filePath)));
                String line = null;
                if (fileName.contains("spms")) {
                    lineStringBuilder.append("spam\t");
                } else {
                    lineStringBuilder.append("ham\t");
                }
                while ((line = br.readLine()) != null) {
                    lineStringBuilder.append(line);
                }
                if (directoryName.equals("part10")) {
                    testList.add(lineStringBuilder.toString());
                } else {
                    spamBufferedWriter.write(lineStringBuilder.toString() + "\n");
                }
                lineStringBuilder.setLength(0);
            }
        }
    }
} catch (IOException ex) {
    // Handle exceptions
    System.out.println("Files can't be found");
}

Files can't be found


In [22]:
// Test the model using the test data in testListArrayList
try (InputStream dataInputStream = new FileInputStream("../data/spamtraining.train")) {
    // Create input stream for training data
    ObjectStream<String> objectStream = new PlainTextByLineStream(dataInputStream, StandardCharsets.UTF_8);
    ObjectStream<DocumentSample> documentSampleStream = new DocumentSampleStream(objectStream);

    DoccatModel documentCategorizationModel = DocumentCategorizerME.train("en", documentSampleStream);
    DocumentCategorizerME documentCategorizer = new DocumentCategorizerME(documentCategorizationModel);
    for (String testItem : testList) {
        double[] probabilities = documentCategorizer.categorize(testItem);
        String bestCategory = documentCategorizer.getBestCategory(probabilities);
        System.out.println("The best fit for: [" + testItem.subSequence(0, 32) + "...] is: " + bestCategory);
    }
} catch (FileNotFoundException ex) {
    // Handle exceptions
} catch (IOException ex) {
    // Handle exceptions
}

CompilationException: 